In [3]:
# First lets get set up!
from os import path
import nexradaws
import time
import tempfile
import os
import shutil
import pyart
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature

from cartopy.feature import NaturalEarthFeature
import matplotlib.colors as colors


import numpy as np

import pytz
from copy import deepcopy

from math import atan2 as atan2
import warnings

%matplotlib inline

#Scale bar code. Because I like scale bars
#This code courtesy and gratefully received from Tim Lang @tjlang on twitter

def gc_latlon_bear_dist(lat1, lon1, bear, dist):
    """
    Input lat1/lon1 as decimal degrees, as well as bearing and distance from
    the coordinate. Returns lat2/lon2 of final destination. Cannot be
    vectorized due to atan2.
    """
    re = 6371.1  # km
    lat1r = np.deg2rad(lat1)
    lon1r = np.deg2rad(lon1)
    bearr = np.deg2rad(bear)
    lat2r = np.arcsin((np.sin(lat1r) * np.cos(dist/re)) +
                      (np.cos(lat1r) * np.sin(dist/re) * np.cos(bearr)))
    lon2r = lon1r + atan2(np.sin(bearr) * np.sin(dist/re) *
                          np.cos(lat1r), np.cos(dist/re) - np.sin(lat1r) *
                          np.sin(lat2r))
    return np.rad2deg(lat2r), np.rad2deg(lon2r)
 
 
 
def add_scale_line(scale, ax, projection, color='k',
                  linewidth=None, fontsize=None, fontweight=None):
    """
    Adds a line that shows the map scale in km. The line will automatically
    scale based on zoom level of the map. Works with cartopy.
 
    Parameters
    ----------
    scale : scalar
        Length of line to draw, in km.
    ax : matplotlib.pyplot.Axes instance
        Axes instance to draw line on. It is assumed that this was created
        with a map projection.
    projection : cartopy.crs projection
        Cartopy projection being used in the plot.
 
    Other Parameters
    ----------------
    color : str
        Color of line and text to draw. Default is black.
    """
    frac_lat = 0.1  # distance fraction from bottom of plot
    frac_lon = 0.5  # distance fraction from left of plot
    e1 = ax.get_extent()
    center_lon = e1[0] + frac_lon * (e1[1] - e1[0])
    center_lat = e1[2] + frac_lat * (e1[3] - e1[2])
    # Main line
    lat1, lon1 = gc_latlon_bear_dist(
        center_lat, center_lon, -90, scale / 2.0)  # left point
    lat2, lon2 = gc_latlon_bear_dist(
        center_lat, center_lon, 90, scale / 2.0)  # right point
    if lon1 <= e1[0] or lon2 >= e1[1]:
        warnings.warn('Scale line longer than extent of plot! ' +
                      'Try shortening for best effect.')
    ax.plot([lon1, lon2], [lat1, lat2], linestyle='-',
            color=color, transform=projection, 
            linewidth=linewidth)
    # Draw a vertical hash on the left edge
    lat1a, lon1a = gc_latlon_bear_dist(
        lat1, lon1, 180, frac_lon * scale / 20.0)  # bottom left hash
    lat1b, lon1b = gc_latlon_bear_dist(
        lat1, lon1, 0, frac_lon * scale / 20.0)  # top left hash
    ax.plot([lon1a, lon1b], [lat1a, lat1b], linestyle='-',
            color=color, transform=projection, linewidth=linewidth)
    # Draw a vertical hash on the right edge
    lat2a, lon2a = gc_latlon_bear_dist(
        lat2, lon2, 180, frac_lon * scale / 20.0)  # bottom right hash
    lat2b, lon2b = gc_latlon_bear_dist(
        lat2, lon2, 0, frac_lon * scale / 20.0)  # top right hash
    ax.plot([lon2a, lon2b], [lat2a, lat2b], linestyle='-',
            color=color, transform=projection, linewidth=linewidth)
    # Draw scale label
    ax.text(center_lon, center_lat - frac_lat * (e1[3] - e1[2]) / 4.0,
            str(int(scale)) + ' km', horizontalalignment='center',
            verticalalignment='center', color=color, fontweight=fontweight,
            fontsize=fontsize)

    
#https://stackoverflow.com/questions/32237862/find-the-closest-date-to-a-given-date/32237949
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))


# This is a VERY LAZY function. 
#No testing and if it gets a file Py-ART can not read it will fail, ungracefully

#Function to pull a radar file from AWS

def get_my_radar(connex, site, this_datetime):
    tlocation = tempfile.mkdtemp()
    these_scans = connex.get_avail_scans(this_datetime.year,this_datetime.month, this_datetime.day, site)
    these_times = [scan.scan_time for scan in these_scans]
    targ = this_datetime
    
    #Need to clean
    these_good_scans = []
    these_good_times = []
    for i in range(len(these_scans)):
        if these_times[i] is not None:
            these_good_times.append(these_times[i])
            these_good_scans.append(these_scans[i])
    
    print(len(these_good_scans), len(these_good_times))

    this_nearest_time = nearest(these_good_times, targ)
    this_index = these_good_times.index(this_nearest_time)
    lcn = tlocation
    localfiles = conn.download(these_good_scans[this_index],lcn)
    return pyart.io.read(localfiles.success[0].filepath)

def get_radar_rain(awsconn, radar_time, radar_site='KLOT',  a=300., b=1.4):
    #Lets just repeat this here:
   
    #It needs to be time zone aware

    #If you want the latest use: datetime.utcnow()
    #And like magic, grab the radar file from the Boston WSR-88D
    radar = get_my_radar(awsconn, radar_site, radar_time)
    #Grab a COPY of the reflectivty data from the radar file. 
    #Convert it from dB to linear units ALWAYS DO THIS!
    z_lin = 10.0**(radar.fields['reflectivity']['data'].copy()/10.)

    #Apply the Z-S relation, in this case the WSR-88D Northeast relation
    rain_z = (z_lin/a)**(1./b) 

    #Add the field back to the radar. Use reflectivity as a template
    radar.add_field_like('reflectivity', 'rain_z',  rain_z, replace_existing = True)

    #Update units and metadata
    radar.fields['rain_z']['units'] = 'mm/h'
    radar.fields['rain_z']['standard_name'] = 'rainfall_rate'
    radar.fields['rain_z']['long_name'] = 'rainfall_rate_from_z'
    radar.fields['rain_z']['valid_min'] = 0
    radar.fields['rain_z']['valid_max'] = 500
    radar.fields['rain_z']['comment'] = "a= %s "%a + 'b=%s'%b
    
    return radar

def do_my_plot_rain(iradar, smax=150, app='', outdir=None, window=2, scb=100.):
    
    #Use this line if you want a log colormap
    norm = None #colors.LogNorm(vmin=0.1, vmax=15)
    
    #Where you want the map centered

    centerlat = 29.5
    centerlon = -95.25
    #delta lat lon degrees
    locbox = [centerlon - window, centerlon + window, centerlat - window, centerlat + window]
    
    #Set up the figure
    myf = plt.figure(figsize=[10,10])
    
    #We are going FULL CUSTOM.. This sets up the map panel
    map_panel_axes = [0.05, 0.05, .8, .8]
    
    #Add the map axes. Wse the Platte Carree projection. 
    ax = myf.add_axes(map_panel_axes, projection=ccrs.PlateCarree())

    #Create a radar display.. this makes Py-ART do a bunch of calcs for radar -> Cartesian coords
    myd = pyart.graph.RadarMapDisplay(iradar)
    
    #Plot a PPI, lowest tilt. Use the Homeyer CVD friendly colormap. Range between zere and smax mm/hr
    #Skip the bells and whistles, we will roll our own (embelish)
    myd.plot_ppi_map('rain_z', 0, vmin=0, vmax=smax, ax=ax, embelish=False, colorbar_flag=False,
                    cmap=pyart.graph.cm_colorblind.HomeyerRainbow,
                    norm=norm)

    #Throw on some gridlines
    gl = ax.gridlines(draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.ylabels_right = False

    #Grab a high res coast
    coast = NaturalEarthFeature(category='physical', scale='10m',
                                edgecolor='black', name='coastline')
    
    counties = NaturalEarthFeature(category='cultural', scale='10m',
                                edgecolor='black', name='admin_2_counties_lakes', alpha=.2)
    #https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_2_counties.zip

    #add the coast and state boundaries
    ax.add_feature(cfeature.STATES, facecolor='none', edgecolor='black')
    ax.add_feature(coast, facecolor='none', edgecolor='black')
    ax.add_feature(counties, facecolor='none', edgecolor='black')
    
    #Zoom the plot
    plt.xlim(locbox[0], locbox[1])
    plt.ylim(locbox[2], locbox[3])

    #Add a scale bar, code at the top of the notebook. Thanks Dr Tim Lang from NASA Marshall
    add_scale_line(scb, ax, projection=ccrs.PlateCarree(), 
                   color='black', linewidth=3,
                  fontsize=20,
                  fontweight='bold')

    #Throw on a colorbar
    cbar = plt.colorbar(mappable=myd.plots[0], fraction=.1, shrink=.8)
    pos = cbar.ax.get_position()
    cbar.ax.set_aspect('auto')
    
    #Ok! Folks from the USA like inches.. So make a twin axis for the colorbar and scale to inches
    cax2 = cbar.ax.twinx()
    cax2.set_ylim([0,smax/25.4])
    pos.x0 +=0.05
    cbar.ax.set_position(pos)
    cax2.set_position(pos)

    #Add the text plus any appended text using app keyword.
    cax2.text(2.3,  .15*smax/25.4,'NEXRAD inst. rain Rate (in/h) ' + app,rotation=90)
    cax2.text(-1.8, .15*smax/25.4,'NEXRAD inst. rain Rate (mm/h) ' + app,rotation=90)
    
    #This deals with filename formatting
    ff = time.strptime(iradar.time['units'][14:], '%Y-%m-%dT%H:%M:%SZ')
    strform = time.strftime('%Y%m%d-%H%M%S',ff)
    print(strform)
    if outdir is None:
        base_file = path.expanduser('~') + '/' + strform #dumps in your home dir
    else:
        base_file = outdir + strform
    
    #Save away! Oh and gets rid of transparency.. Tranparency makes animated gifs look odd
    plt.savefig(base_file+'rain.png', facecolor='white', transparent=False)

In [16]:
conn = nexradaws.NexradAwsInterface()

In [18]:
start_event_time = datetime(2022, 3, 22, 10, 10, 0).replace(tzinfo=pytz.UTC)

#tdelt = 0

for i in range(10):
    tdelt = tdelt + 10
    event_time = start_event_time + timedelta(minutes=tdelt)
    KLCH_radar = get_radar_rain(conn, event_time, radar_site='KLCH')
    KHGX_radar = get_radar_rain(conn, event_time, radar_site='KHGX')

    ff = time.strptime(KHGX_radar.time['units'][14:], '%Y-%m-%dT%H:%M:%SZ')
    strform = time.strftime('%Y%m%d-%H%M%S',ff)
    print(strform)

    khgx_f = pyart.filters.GateFilter(KHGX_radar)
    khgx_f.exclude_masked('reflectivity')
    khgx_f.exclude_invalid('reflectivity')

    klch_f = pyart.filters.GateFilter(KLCH_radar)
    klch_f.exclude_masked('reflectivity')
    klch_f.exclude_invalid('reflectivity')
    pyart.io.write_cfradial('/Users/scollis/data/tracer_20220322/KHGX_radial-'+strform+'.nc', KHGX_radar)
    pyart.io.write_cfradial('/Users/scollis/data/tracer_20220322/KLCH_radial-'+strform+'.nc', KLCH_radar)

    grids = pyart.map.grid_from_radars((KHGX_radar, KLCH_radar),(31,101,101),
                       ((0.,15000.),(-100000.,100000.),(-100000.,100000.)), 
                                       weighting_function='Barnes2',
                                      fields=['reflectivity','rain_z'],
                                      gatefilters=(khgx_f, klch_f))
    xgrids = grids.to_xarray()
    xgrids.to_netcdf('/Users/scollis/data/tracer_20220322/yyy-grids-'+strform+'.nc')

    #pyart.io.write_grid('/Users/scollis/data/tracer_20220322/xgrids-'+strform+'.nc', grids)


234 234
Downloaded KLCH20220322_115933_V06
1 out of 1 files downloaded...0 errors
246 246
Downloaded KHGX20220322_120125_V06
1 out of 1 files downloaded...0 errors
20220322-120125
234 234
Downloaded KLCH20220322_121103_V06
1 out of 1 files downloaded...0 errors
246 246
Downloaded KHGX20220322_120749_V06
1 out of 1 files downloaded...0 errors
20220322-120749
234 234
Downloaded KLCH20220322_121732_V06
1 out of 1 files downloaded...0 errors
246 246
Downloaded KHGX20220322_122027_V06
1 out of 1 files downloaded...0 errors
20220322-122027
234 234
Downloaded KLCH20220322_122919_V06
1 out of 1 files downloaded...0 errors
246 246
Downloaded KHGX20220322_122651_V06
1 out of 1 files downloaded...0 errors
20220322-122651
234 234
Downloaded KLCH20220322_124231_V06
1 out of 1 files downloaded...0 errors
246 246
Downloaded KHGX20220322_123940_V06
1 out of 1 files downloaded...0 errors
20220322-123940
235 235
Downloaded KLCH20220322_124835_V06
1 out of 1 files downloaded...0 errors
246 246
Downloaded

In [8]:
timedelta?